<a href="https://colab.research.google.com/github/audalsgh/20250729/blob/main/0729_peoplenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yt-dlp
!pip install onnxruntime-gpu  # GPU 지원
!pip install opencv-python-headless numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.7 MB/s eta 0:00:00


In [4]:
# 직접 업로드
from google.colab import files
uploaded = files.upload()  # 여기서 peoplenet.onnx 선택

Saving resnet34_peoplenet_int8.onnx to resnet34_peoplenet_int8.onnx


In [7]:
import onnxruntime as ort
import cv2
import numpy as np
import glob

# 모델 경로 지정
model_path = '/content/resnet34_peoplenet_int8.onnx'  # (A)의 경우 업로드한 파일명
# model_path = '/content/drive/MyDrive/models/peoplenet.onnx'  # (B) Drive 경로

# ONNX InferenceSession 생성 (CUDA, fallback to CPU)
providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
session = ort.InferenceSession(model_path, providers=providers)

# 입력 이름 확인
input_name = session.get_inputs()[0].name
print(f"✅ ONNX 모델 로드 완료. Input Name: {input_name}")

✅ ONNX 모델 로드 완료. Input Name: input_1:0


In [9]:
# 1) yt-dlp 설치 (이미 설치했다면 스킵)
!pip install -q yt-dlp

# 2) 유튜브 영상 다운로드
import yt_dlp

YOUTUBE_URL = 'https://www.youtube.com/watch?v=J71x_FW1Mdg'
OUTPUT_FILENAME = 'video.mp4'

ydl_opts = {
    'format': 'mp4',
    'outtmpl': OUTPUT_FILENAME,
    # 필요시 쿠키나 헤더 옵션 추가 가능
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    print(f"▶️ 다운로드 시작: {YOUTUBE_URL}")
    ydl.download([YOUTUBE_URL])
    print(f"✅ 다운로드 완료: {OUTPUT_FILENAME}")

# 3) video_path 변수에 할당
video_path = OUTPUT_FILENAME
print(f"🎬 사용할 비디오 경로: {video_path}")

▶️ 다운로드 시작: https://www.youtube.com/watch?v=J71x_FW1Mdg
[youtube] Extracting URL: https://www.youtube.com/watch?v=J71x_FW1Mdg
[youtube] J71x_FW1Mdg: Downloading webpage
[youtube] J71x_FW1Mdg: Downloading tv client config
[youtube] J71x_FW1Mdg: Downloading player 662eb823-main
[youtube] J71x_FW1Mdg: Downloading tv player API JSON
[youtube] J71x_FW1Mdg: Downloading ios player API JSON
[youtube] J71x_FW1Mdg: Downloading m3u8 information
[info] J71x_FW1Mdg: Downloading 1 format(s): 18
[download] Destination: video.mp4
[download] 100% of    5.49MiB in 00:00:01 at 4.65MiB/s   
✅ 다운로드 완료: video.mp4
🎬 사용할 비디오 경로: video.mp4


In [21]:
# 4) ONNX 입력 메타데이터로 기대 입력 크기 확인
input_meta = session.get_inputs()[0]
_, C, H, W = input_meta.shape
print(f"Model expects input shape (N, C, H, W): (1, {C}, {H}, {W})")

# 5) OpenCV로 비디오 열기 및 VideoWriter 초기화
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError(f"동영상을 열 수 없습니다: {video_path}")

fps    = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out    = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

print(f"▶️ 추론 시작: fps={fps}, size=({width},{height})")

# 6) 프레임별 추론 및 비디오 기록 (전체 채널 합산 + closing)
import cv2
import numpy as np

frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 전처리
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (W, H))
    img = img.astype(np.float32) / 255.0
    img = np.transpose(img, (2,0,1))[None, ...]

    # ONNX 추론
    outputs = session.run(None, {input_meta.name: img})
    heatmaps = outputs[0][0]           # (3, H, W)

    # ❗️ 모든 채널 합산 (head+torso+legs)
    heatmap = np.sum(heatmaps, axis=0)  # (H, W)

    # 원본 프레임 크기에 맞춰 리사이즈 & 정규화(0~255) & uint8
    mask = cv2.resize(heatmap, (width, height))
    mask_uint8 = (mask * 255).clip(0,255).astype(np.uint8)

    # Morphological closing으로 작은 구멍 메우기
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
    mask_closed = cv2.morphologyEx(mask_uint8, cv2.MORPH_CLOSE, kernel)

    # 이진 임계처리 (값 분포에 따라 100~150 사이로 조절)
    _, bin_mask = cv2.threshold(mask_closed, 120, 255, cv2.THRESH_BINARY)

    # 컨투어 추출 및 박스 그리기
    contours, _ = cv2.findContours(bin_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        if w*h < 500:  # 노이즈 필터
            continue
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

    # 프레임 번호 표시
    cv2.putText(frame,
                f"Frame {frame_idx}",
                (10,30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1, (0,255,0), 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print(f"✅ 처리 완료: output.mp4 (총 {frame_idx} 프레임)")

# 7) 결과 비디오 다운로드
from google.colab import files
files.download('output.mp4')

Model expects input shape (N, C, H, W): (1, 3, 544, 960)
▶️ 추론 시작: fps=29.97002997002997, size=(640,360)
✅ 처리 완료: output.mp4 (총 2786 프레임)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>